In [35]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)

from models import *
from backbones.config import config
import pathlib

In [38]:


print(config)
for k, v in config.items():
  print("{} : {}".format(k, v))
  pathlib.Path('../figures/{}'.format(k)).mkdir(parents=True, exist_ok=True) 

m_li = [
  # base_resnet50(model_path=config['resnet50'], num_classes=80, image_size=224, pretrained=True),\
  #  base_vit(config['vit'], 80, image_size=224, pretrained=True),\
      base_swin(config['swin_large'], 80, image_size=384, pretrained=True),\
        #  base_convnext(config['convnext'], 80, image_size=224, pretrained=True), \
          #  base_mlpmixer(config['mlpmixer'], num_classes=80, image_size=224, pretrained=True), \
             base_resnet101(model_path=config['resnet101'], num_classes=80, image_size=448, pretrained=True)
             ]
# m_li2 = [BaseResnet(m_li[0], 80)]
p_li = [
  # '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_resnet50_base_best.pth.tar', \
  # '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_vit_base_best.pth.tar', \
    # '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_swin_base_best.pth.tar',\
  # '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in22k-4-4-0_convnext_base_best.pth.tar',\
      # '/home/seongha/LT-ML/checkpoint/coco/coco_LT(0)_label_cnt_in21k-4-4-0_mlpmixer_base_best.pth.tar' ,\
      'checkpoint/coco/coco_bs50-384-scheduler_swin_large_base_best.pth.tar',\
        '../checkpoint/coco/coco_bs64-448-scheduler_resnet101_base_best.pth.tar  '
  ]

def get_model(index):
  path = p_li[index]
  model = m_li[index]
  di = torch.load(path)
  print(di['best_score'])
  print(di.keys())
  model.load_state_dict(di['state_dict'])

  for n, p in model.named_parameters():
    if p.requires_grad==False:
      p.requires_grad=True
  return model


{'resnet50': 'resnet50', 'swin': 'swin_base_patch4_window12_384_in22k', 'swin_large': 'swin_large_patch4_window12_384_in22k', 'convnext': 'convnext_large_in22k', 'resnet101': 'resnetv2_101x1_bitm_in21k'}
resnet50 : resnet50
swin : swin_base_patch4_window12_384_in22k
swin_large : swin_large_patch4_window12_384_in22k
convnext : convnext_large_in22k
resnet101 : resnetv2_101x1_bitm_in21k
InterSwin
2304 384


Examples

In [ ]:
# pool with window of size=3, stride=2
m = nn.AvgPool1d(3, stride=2)
m(torch.tensor([[[1.,2,3,4,5,6,7]]])) #1,1,7

In [5]:
# pool of square window of size=3, stride=2
m = nn.AvgPool2d(3, stride=2)
# pool of non-square window
m = nn.AvgPool2d((63, 63), stride=(1, 1))
input = torch.randn(20, 768, 63, 63)
output = m(input)
output.shape

torch.Size([20, 768, 1, 1])

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=768, nhead=1)
memory = torch.rand(32, 196, 768)
m = nn.AvgPool2d((63, 63), stride=(1, 1))
memory = m(memory)
memory = torch.squeeze(memory, -1)
memory = torch.squeeze(memory, -1)
tgt = torch.rand(32, 768)
out = decoder_layer(tgt, memory) #32,768

m = nn.AvgPool1d(196, stride=1)
out = m(out)
out.shape

In [ ]:

# Calculate symmetric padding for a convolution
def get_padding(kernel_size: int, stride: int = 1, dilation: int = 1, **_) -> int:
    padding = ((stride - 1) + dilation * (kernel_size - 1)) // 2
    return padding


In [46]:
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
# # non-square kernels and unequal stride and with padding
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# # non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(256, 2048, (2, 2), stride=(171, 171), dilation=(2, 2), padding=1)
input = torch.randn(20, 256, 122, 122)
output = m(input)
output.shape

torch.Size([20, 2048, 1, 1])

In [1]:
# pool of square window of size=3, stride=2
m = nn.AvgPool2d(3, stride=2)
# pool of non-square window
m = nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
output.shape

NameError: name 'nn' is not defined

In [11]:
#swin
model = get_model(2)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
print(out.shape)

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1], model.features[2][0], model.features[2][1]]) #2-2-17-2 blocks
out = seq(inp)
print(out.shape) #[1, 784, 256], [1, 196, 512]
b, n, h = out.shape
out = out.reshape((1, -1))

m = nn.AvgPool1d(n*h - 1024 + 1, stride=1)
out = m(out).squeeze(-1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

{'mAP': tensor(74.8108), 'OF1': 0.7193339297047961, 'CF1': 0.6759560773634149}
dict_keys(['epoch', 'arch', 'state_dict', 'best_score'])
torch.Size([1, 1024])
torch.Size([1, 196, 512])
torch.Size([1, 1024])
torch.Size([1, 80])


In [ ]:
#vit
model = get_model(1)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
print(out.shape)

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1], model.features[2][0], model.features[2][1], model.features[2][2]])
out = seq(inp)
print(out.shape) #([1, 196, 768])
b, n, h = out.shape

out = torch.swapaxes(out, 1,2)
m = nn.AvgPool1d(196, stride=1)
out = m(out).squeeze(-1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

In [40]:
#resnet50
import timm
model = timm.create_model("resnet50", pretrained=True, num_classes=80)
inp = torch.rand(3, 488, 488)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.conv1, model.bn1, model.act1, model.maxpool, model.layer1, model.layer2, model.layer3, model.layer4])
out = seq(inp)
print(out.shape) #torch.Size([1, 2048, 16, 16])

inp = torch.rand(3, 488, 488)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.conv1, model.bn1, model.act1, model.maxpool, model.layer1])
out = seq(inp)
print(out.shape) #[1, 256, 122, 122])
b, n, h, w = out.shape
out = out.reshape((1, -1))
print(out.shape) #3810304

m = nn.AvgPool1d(n*h*w - 3810304 + 1, stride=1)
# m = nn.Conv2d(n, )
out = m(out)
out = out.reshape((-1, 6144, 1, 1))
print(out.shape)

logit = model.head.fc(out)
print(logit.shape)
logit = logit.flatten()
print(logit.shape)

AttributeError: 'ResNetV2' object has no attribute 'conv1'

In [39]:
#resnet101
model = get_model(-1)
inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
out = model.features(inp)
out = model.head.global_pool(out)
print(out.shape) #torch.Size([1, 6144, 1, 1])

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0], model.features[1][1],model.features[1][2]])
out = seq(inp)
print(out.shape) #[1, 3072, 14, 14])
b, n, h, w = out.shape
out = out.reshape((1, -1))
print(out.shape)

m = nn.AvgPool1d(n*h*w - 6144 + 1, stride=1)
# m = nn.Conv2d(n, )
out = m(out)
out = out.reshape((-1, 6144, 1, 1))
print(out.shape)

logit = model.head.fc(out)
print(logit.shape)
logit = logit.flatten()
print(logit.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../checkpoint/coco/coco_bs64-448-scheduler_resnet101_base_best.pth.tar  '

mlpmixer

In [ ]:
model = get_model(4)
# import torch.nn.functional as F

inp = torch.rand(3, 224, 224)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0]])
out = seq(inp)
print(out.shape)
out = torch.swapaxes(out, 1,2)
m = nn.AvgPool1d(196, stride=1)
out = m(out).squeeze(-1)
# out = torch.mean(out.view(out.size(0), out.size(1), -1), dim=2)
torch.mean(torch.stack([out, out]), 1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

In [7]:
model = get_model(4)
# import torch.nn.functional as F

inp = torch.rand(3, 448, 448)
inp = torch.unsqueeze(inp, 0)
seq = nn.Sequential(*[model.features[0], model.features[1][0]])
out = seq(inp)
print(out.shape)
out = torch.swapaxes(out, 1,2)
m = nn.AvgPool1d(196, stride=1)
out = m(out).squeeze(-1)
# out = torch.mean(out.view(out.size(0), out.size(1), -1), dim=2)
torch.mean(torch.stack([out, out]), 1)
print(out.shape)

logit = model.fc(out)
print(logit.shape)

NameError: name 'get_model' is not defined

## Count number of training parameters

In [4]:
def count_training_params(model, path):
  print(model.__class__)
  di = torch.load(path)
  model.load_state_dict(di['state_dict'])
  ##full finetune
  print(len(list(model.parameters())))

  ## ours
  print(np.sum(np.array(list(p.requires_grad for p in model.parameters()))))
  for n, p in model.named_parameters():
    if p.requires_grad:
      print(n)

  ## linear probe
  cnt=0
  for n, p in model.fc.named_parameters():
    if p.requires_grad:
      cnt+=1
  print(cnt)

model = base_swin(config['swin_large'], 80, 384, True, cond=True, where=0)
path='../checkpoint/coco/coco_bs50-at0-lr_scheduler_swin_large_base_best.pth.tar'
count_training_params(model,path)
path='../checkpoint/coco/coco_@0-bs64-448-scheduler_resnet101_base_best.pth.tar'
model = base_resnet101(config['resnet101'], 80, 448, True, cond=True, where=0)
count_training_params(model,path)

/home/seongha/anaconda3/envs/LTML/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


InterSwin
2304 384
<class 'backbones.swin.InterSwin'>
330
3
scale
fc.weight
fc.bias
2
<class 'backbones.resnet.InterResnetV2'>
307
3
scale
fc.weight
fc.bias
2
